In [145]:
df = pd.read_csv('trails_df.csv')

In [76]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

class PersonalizedTrailRecommender:
    def __init__(self):
        self.trails_df = None
        self.mlb = MultiLabelBinarizer()

    def preprocess_trails(self, trails_data):
        """Preprocess the trail data."""
        self.trails_df = pd.DataFrame(trails_data)
        self.trails_df['Tags'] = self.trails_df['Tags'].str.split(', ')

    def process_user_preferences(self, user_response):
        """Convert user preferences into feature vectors."""
        features = {}

        # Activity level mapping
        activity_map = {
            'Pretty active 💪': 'hard',
            'Average, but could be better 🚶': 'moderate',
            'Not my thing': 'easy'
        }

        # Trail preference mapping
        trail_preferences = {
            'Thrilling and tough mountain trails 🗻': ['Rocky', 'Scramble', 'Hard'],
            'Easy paths with good views 🗾': ['Views', 'Easy'],
            'Just a relaxing walk through the woods 🌲': ['Forest', 'Walking', 'Easy']
        }

        # Weather preference mapping
        weather_prefs = {
            'Rain won\'t stop me! 🌧': ['all_weather'],
            'Maybe if it\'s just cloudy ☁': ['fair_weather'],
            'No way, I\'ll reschedule 🛌': ['fair_weather']
        }

        # Convert user's activity level
        activity_level = activity_map.get(user_response.get('activity_level', 'Average, but could be better 🚶'))
        features['preferred_difficulty'] = 1 if activity_level == 'hard' else 0.5 if activity_level == 'moderate' else 0

        # Process trail preferences
        preferred_tags = []
        for pref in trail_preferences:
            if pref in user_response.get('trail_preference', []):
                preferred_tags.extend(trail_preferences[pref])

        # Add weather tolerance
        weather_pref = user_response.get('weather_preference', 'Maybe if it\'s just cloudy ☁')
        features['weather_tolerance'] = 1 if weather_prefs.get(weather_pref, ['fair_weather'])[0] == 'all_weather' else 0.5

        # Add experience level
        experience_map = {
            'Yes, a few times': 0.8,
            'No, but I\'d like to try': 0.4,
            'Not my thing': 0.2
        }
        features['experience'] = experience_map.get(user_response.get('hiking_experience', 'No, but I\'d like to try'), 0.4)

        return features, preferred_tags

    def get_recommendations(self, user_response, n_recommendations=3):
        """Get personalized trail recommendations based on user preferences."""
        features, preferred_tags = self.process_user_preferences(user_response)

        # Calculate trail scores
        scores = []
        for _, trail in self.trails_df.iterrows():
            score = 0

            # Difficulty match
            if trail['Difficulty'] == 'Hard':
                score += features['preferred_difficulty'] * 2
            elif trail['Difficulty'] == 'Moderate':
                score += (1 - abs(0.5 - features['preferred_difficulty'])) * 1.5
            else:
                score += (1 - features['preferred_difficulty']) * 1

            # Tag match
            trail_tags = set(trail['Tags'])
            preferred_tags_set = set(preferred_tags)
            tag_match = len(trail_tags.intersection(preferred_tags_set)) / len(preferred_tags_set) if preferred_tags_set else 0.5
            score += tag_match * 2

            # Experience level consideration
            if trail['Difficulty'] == 'Hard' and features['experience'] < 0.6:
                score *= 0.7

            # Weather consideration
            if 'all_weather' in trail_tags:
                score *= (1 + features['weather_tolerance'])

            scores.append(score)

        # Get top recommendations
        recommendations_idx = np.argsort(scores)[::-1][:n_recommendations]
        recommendations = self.trails_df.iloc[recommendations_idx].copy()
        recommendations['Match_Score'] = [scores[i] for i in recommendations_idx]

        return recommendations

def format_user_recommendations(recommendations):
    """Format the recommendations in a readable way."""
    output = []
    for _, trail in recommendations.iterrows():
        output.append(
            f"🏃‍♂️ {trail['Trail_name']}\n"
            f"📍 Location: {trail['Location']}\n"
            f"💪 Difficulty: {trail['Difficulty']}\n"
            f"📏 {trail['Length']}\n"
            f"⭐ Rating: {trail['Average_rating']}/5.0\n"
            f"🎯 Match Score: {trail['Match_Score']:.2f}\n"
            f"🏷️ Features: {', '.join(trail['Tags'])}\n"
        )
    return "\n".join(output)

# Example usage for a user:
sample_user = {
    'name': 'Jyotin Goel',
    'location': 'Delhi',
    'activity_level': 'Pretty active 💪',
    'hiking_experience': 'No, but I\'d like to try',
    'trail_preference': ['Thrilling and tough mountain trails 🗻'],
    'weather_preference': 'Rain won\'t stop me! 🌧'
}

recommender = PersonalizedTrailRecommender()
# Preprocess trails data first
recommender.preprocess_trails(df)
# Get personalized recommendations
recommendations = recommender.get_recommendations(sample_user)
# Print formatted recommendations
print(format_user_recommendations(recommendations))

🏃‍♂️ Kanamo West Peak Trail
📍 Location: Kibber Wildlife Sanctuary
💪 Difficulty: Hard
📏 Length: 19.8 km
⭐ Rating: 5.0/5.0
🎯 Match Score: 2.33
🏷️ Features: Camping, Bird watching, Hiking, Snowshoeing, Backpacking, Lake, River, Views, Waterfall, Scramble, Rocky, Snow

🏃‍♂️ Prinkti La (Shilla - Lamayuru)
📍 Location: Khaltse, Ladakh, India
💪 Difficulty: Hard
📏 Length: 9.5 km
⭐ Rating: 5.0/5.0
🎯 Match Score: 2.33
🏷️ Features: Hiking, Snowshoeing, Walking, River, Views, Wildflowers, Scramble, Rocky, No shade

🏃‍♂️ Great Himalayan Lakes
📍 Location: Khag, Jammu and Kashmir, India
💪 Difficulty: Hard
📏 Length: 67.8 km
⭐ Rating: 5.0/5.0
🎯 Match Score: 2.33
🏷️ Features: Camping, Bird watching, Hiking, Walking, Backpacking, Forest, Lake, River, Views, Wildflowers, Wildlife, Scramble, Rocky, Snow



In [77]:
recommender = PersonalizedTrailRecommender()
# Preprocess trails data first
recommender.preprocess_trails(df)
# Get personalized recommendations
recommendations = recommender.get_recommendations(sample_user)
# Print formatted recommendations
print(format_user_recommendations(recommendations))

🏃‍♂️ Kanamo West Peak Trail
📍 Location: Kibber Wildlife Sanctuary
💪 Difficulty: Hard
📏 Length: 19.8 km
⭐ Rating: 5.0/5.0
🎯 Match Score: 2.33
🏷️ Features: Camping, Bird watching, Hiking, Snowshoeing, Backpacking, Lake, River, Views, Waterfall, Scramble, Rocky, Snow

🏃‍♂️ Prinkti La (Shilla - Lamayuru)
📍 Location: Khaltse, Ladakh, India
💪 Difficulty: Hard
📏 Length: 9.5 km
⭐ Rating: 5.0/5.0
🎯 Match Score: 2.33
🏷️ Features: Hiking, Snowshoeing, Walking, River, Views, Wildflowers, Scramble, Rocky, No shade

🏃‍♂️ Great Himalayan Lakes
📍 Location: Khag, Jammu and Kashmir, India
💪 Difficulty: Hard
📏 Length: 67.8 km
⭐ Rating: 5.0/5.0
🎯 Match Score: 2.33
🏷️ Features: Camping, Bird watching, Hiking, Walking, Backpacking, Forest, Lake, River, Views, Wildflowers, Wildlife, Scramble, Rocky, Snow



In [111]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

class PersonalizedTrailRecommender:
    def __init__(self):
        self.trails_df = None
        self.mlb = MultiLabelBinarizer()

    def preprocess_trails(self, trails_data):
        """Preprocess trail data to clean and structure it."""
        self.trails_df = pd.DataFrame(trails_data)
        self.trails_df['Tags'] = self.trails_df['Tags'].str.split(', ')
        # Extract numeric part of length and convert to float
        self.trails_df['Length'] = self.trails_df['Length'].str.extract(r'([\d.]+)').astype(float)

    def process_user_preferences(self, user_response):
        """Convert user preferences into feature vectors and tags."""
        features = {}
        preferred_tags = []

        # Mappings for user preferences to tags and features
        mappings = {
            'home_environment': {
                'A big city 🌆': ['Urban', 'Accessible'],
                'A peaceful small town 🏡': ['Accessible', 'Quiet'],
                'A bustling suburb just outside the city 🏘': ['Urban', 'Accessible'],
                'A rural area surrounded by nature 🌾': ['Nature', 'Rural'],
                'A mountain or hilly area 🏔': ['Mountain', 'Elevation'],
                'A coastal area by the beach 🌊': ['Coastal', 'Beach'],
            },
            'weekend_preference': {
                'Chilling with friends': ['Social', 'Easy'],
                'Exploring new places': ['Adventure', 'Scenic'],
                'Watching movies or gaming': ['Easy', 'Beginner'],
                'Going on adventures (like hiking)': ['Adventure', 'Challenge'],
            },
            'trail_length_preference': {
                'Short and sweet (under 5 km)': ['Short', 'Easy'],
                'A nice challenge (5-10 km)': ['Medium', 'Moderate'],
                'Long and tough (over 10 km)': ['Long', 'Hard'],
            },
            'hiking_companion': {
                'Alone for some peace and quiet': ['Solo', 'Quiet'],
                'With friends or a group': ['Group', 'Social'],
                'Family trips': ['Family', 'Easy'],
                'My pet 🐕': ['Pet-friendly', 'Accessible'],
            },
            'perfect_trail_features': {
                'Great scenery 🌄': ['Scenic', 'Views'],
                'Not too difficult, but enough exercise 💦': ['Moderate', 'Exercise'],
                'Close to a cool destination (like a waterfall or viewpoint) 💧': ['Destination', 'Feature'],
                'Good weather ☀️': ['Fair_weather'],
            },
            'dream_destination': {
                'A tropical island 🏝': ['Tropical', 'Coastal'],
                'A snowy mountain 🏔': ['Mountain', 'Snow'],
                'A peaceful forest 🌳': ['Forest', 'Peaceful'],
                'Anywhere with breathtaking views!': ['Scenic', 'Views'],
            },
            'music_preference': {
                'Chill acoustic tunes 🎸': ['Moderate', 'Peaceful'],
                'Pumped-up workout beats 🎧': ['Hard', 'Challenge'],
                'Nature sounds 🌿': ['Nature', 'Quiet'],
                'No music, just the sound of nature 🐦': ['Nature', 'Peaceful'],
            }
        }

        # Feature mappings for numeric values
        fitness_map = {'Couch potato 🛋': 0.2, 'Average, but could be better 🚶': 0.5, 'Pretty active 💪': 0.8, 'Athlete level 🏃‍♀️': 1.0}
        weather_map = {'No way, I\'ll reschedule 🛌': 0.2, 'Maybe if it\'s just cloudy ☁️': 0.5, 'Rain won\'t stop me! 🌧': 1.0}
        experience_map = {'Yes, a few times': 0.6, 'No, but I\'d like to try': 0.3, 'Yes, and I hike regularly': 1.0, 'Not my thing': 0.1}

        # Extract numeric features
        features['fitness_level'] = fitness_map.get(user_response.get('fitness_level', 'Average, but could be better 🚶'), 0.5)
        features['weather_tolerance'] = weather_map.get(user_response.get('weather_preference', 'Maybe if it\'s just cloudy ☁️'), 0.5)
        features['experience'] = experience_map.get(user_response.get('hiking_experience', 'No, but I\'d like to try'), 0.3)
        features['hiking_likelihood'] = int(user_response.get('hiking_likelihood', 3)) / 5.0

        # Collect preferred tags from mappings
        for category, tag_mapping in mappings.items():
            user_choice = user_response.get(category)
            if isinstance(user_choice, list):
                for choice in user_choice:
                    preferred_tags.extend(tag_mapping.get(choice, []))
            elif user_choice in tag_mapping:
                preferred_tags.extend(tag_mapping[user_choice])

        return features, list(set(preferred_tags))

    def get_recommendations(self, user_response, n_recommendations=3):
        """Get personalized trail recommendations based on user preferences."""
        features, preferred_tags = self.process_user_preferences(user_response)

        # Calculate trail scores
        scores = []
        for _, trail in self.trails_df.iterrows():
            score = 0

            # Match difficulty with fitness and experience
            difficulty_score = (features['fitness_level'] + features['experience']) / 2
            if trail['Difficulty'] == 'Hard':
                score += difficulty_score * 2 if difficulty_score > 0.6 else difficulty_score * 0.5
            elif trail['Difficulty'] == 'Moderate':
                score += difficulty_score * 1.5
            else:
                score += (1 - difficulty_score) * 1.5

            # Tag matching
            trail_tags = set(trail['Tags'])
            tag_match = len(trail_tags.intersection(preferred_tags)) / max(len(preferred_tags), 1)
            score += tag_match * 3

            # Length adjustment
            trail_length = trail['Length']
            if trail_length <= 5:
                score *= 1.2 if 'Short' in preferred_tags else 0.9
            elif trail_length <= 10:
                score *= 1.2 if 'Medium' in preferred_tags else 0.9
            else:
                score *= 1.2 if 'Long' in preferred_tags else 0.8

            # Weather adjustment
            if 'all_weather' in trail_tags:
                score *= (1 + features['weather_tolerance'])
            elif 'fair_weather' in trail_tags and features['weather_tolerance'] < 0.5:
                score *= 1.1

            # Hiking likelihood impact
            score *= (0.5 + features['hiking_likelihood'])
            scores.append(score)

        # Get top recommendations
        recommendations_idx = np.argsort(scores)[::-1][:n_recommendations]
        recommendations = self.trails_df.iloc[recommendations_idx].copy()
        recommendations['Match_Score'] = [scores[i] for i in recommendations_idx]

        return recommendations


def format_user_recommendations(recommendations):
    """Format the recommendations for display."""
    output = []
    for _, trail in recommendations.iterrows():
        output.append(
            f"🏃‍♂️ {trail['Trail_name']}\n"
            f"📍 Location: {trail['Location']}\n"
            f"💪 Difficulty: {trail['Difficulty']}\n"
            f"📏 Length: {trail['Length']} km\n"
            f"⭐ Rating: {trail['Average_rating']}/5.0\n"
            f"🎯 Match Score: {trail['Match_Score']:.2f}\n"
            f"🏷️ Features: {', '.join(trail['Tags'])}\n"
        )
    return "\n".join(output)


# Example usage
sample_user = {
    'home_environment': 'A peaceful small town 🏡',
    'weekend_preference': 'Going on adventures (like hiking)',
    'fitness_level': 'Pretty active 💪',
    'hiking_experience': 'Yes, a few times',
    'trail_length_preference': ['A nice challenge (5-10 km)', 'Long and tough (over 10 km)'],
    'hiking_companion': 'With friends or a group',
    'perfect_trail_features': ['Great scenery 🌄', 'Close to a cool destination (like a waterfall or viewpoint) 💧'],
    'dream_destination': 'A snowy mountain 🏔',
    'music_preference': 'Nature sounds 🌿',
    'weather_preference': 'Rain won\'t stop me! 🌧',
    'hiking_likelihood': 4
}

recommender = PersonalizedTrailRecommender()
recommender.preprocess_trails(df)
recommendations = recommender.get_recommendations(sample_user, n_recommendations=3)
print(format_user_recommendations(recommendations))

🏃‍♂️ Kanamo West Peak Trail
📍 Location: Kibber Wildlife Sanctuary
💪 Difficulty: Hard
📏 Length: 19.8 km
⭐ Rating: 5.0/5.0
🎯 Match Score: 2.73
🏷️ Features: Camping, Bird watching, Hiking, Snowshoeing, Backpacking, Lake, River, Views, Waterfall, Scramble, Rocky, Snow

🏃‍♂️ Auli - Gurson Bugyal
📍 Location: Jyotirmath, Uttarakhand, India
💪 Difficulty: Hard
📏 Length: 13.0 km
⭐ Rating: 4.5/5.0
🎯 Match Score: 2.73
🏷️ Features: Camping, Bird watching, Hiking, Snowshoeing, Walking, Backpacking, Forest, Lake, Views, Wildflowers, Wildlife, Scramble, Rocky, Snow

🏃‍♂️ Waichin Valley
📍 Location: Kullu, Himachal Pradesh, India
💪 Difficulty: Hard
📏 Length: 12.6 km
⭐ Rating: 4.3/5.0
🎯 Match Score: 2.73
🏷️ Features: Camping, Bird watching, Hiking, Snowshoeing, Backpacking, Forest, River, Views, Wildflowers, Wildlife, Scramble, Snow



In [144]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.layers import Dense, Dropout, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

class AdvancedTrailRecommender:
    def __init__(self, embedding_dim=32):
        self.trails_df = None
        self.mlb = MultiLabelBinarizer()
        self.scaler = MinMaxScaler()
        self.embedding_dim = embedding_dim
        self.user_model = None
        self.trail_model = None
        self.hybrid_model = None
        self.tag_embeddings = None

    def preprocess_trails(self, trails_data):
        """Preprocess trail data with advanced feature engineering."""
        self.trails_df = pd.DataFrame(trails_data)

        # Convert tags to binary matrix
        tag_matrix = self.mlb.fit_transform(self.trails_df['Tags'].str.split(', '))

        # Create numeric features
        self.trails_df['Length'] = self.trails_df['Length'].str.extract(r'([\d.]+)').astype(float)
        self.trails_df['Difficulty_Score'] = self.trails_df['Difficulty'].map({
            'Easy': 0.33, 'Moderate': 0.66, 'Hard': 1.0
        })

        # Scale numeric features
        # numeric_features = ['Length', 'Difficulty_Score', 'Average_rating']
        # self.trails_df[numeric_features] = self.scaler.fit_transform(
        #     self.trails_df[numeric_features]
        # )

        # Create trail embeddings using tag information
        self.tag_embeddings = self._create_tag_embeddings(tag_matrix)

    def _create_tag_embeddings(self, tag_matrix):
        """Create trail embeddings using autoencoder."""
        input_dim = tag_matrix.shape[1]

        # Encoder
        inputs = Input(shape=(input_dim,))
        encoded = Dense(64, activation='relu')(inputs)
        encoded = Dense(self.embedding_dim, activation='relu')(encoded)

        # Decoder
        decoded = Dense(64, activation='relu')(encoded)
        decoded = Dense(input_dim, activation='sigmoid')(decoded)

        # Train autoencoder
        autoencoder = Model(inputs, decoded)
        encoder = Model(inputs, encoded)

        autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
        autoencoder.fit(tag_matrix, tag_matrix, epochs=50, batch_size=32, verbose=0)

        return encoder.predict(tag_matrix)

    def _build_neural_models(self, user_feature_dim, trail_feature_dim):
        """Build neural networks for user and trail features."""
        # User model
        user_input = Input(shape=(user_feature_dim,))
        user_dense = Dense(64, activation='relu')(user_input)
        user_dropout = Dropout(0.3)(user_dense)
        user_embedding = Dense(self.embedding_dim, activation='relu')(user_dropout)
        self.user_model = Model(user_input, user_embedding)

        # Trail model
        trail_input = Input(shape=(trail_feature_dim,))
        trail_dense = Dense(64, activation='relu')(trail_input)
        trail_dropout = Dropout(0.3)(trail_dense)
        trail_embedding = Dense(self.embedding_dim, activation='relu')(trail_dropout)
        self.trail_model = Model(trail_input, trail_embedding)

        # Hybrid model
        combined = Concatenate()([user_embedding, trail_embedding])
        dense1 = Dense(8, activation='relu')(combined)
        dropout1 = Dropout(0.3)(dense1)
        dense2 = Dense(4, activation='relu')(dropout1)
        output = Dense(1, activation='sigmoid')(dense2)

        self.hybrid_model = Model(
            [user_input, trail_input],
            output
        )
        self.hybrid_model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

    def process_user_preferences(self, user_response):
        """Process user preferences into feature vector."""
        # Expanded mappings from original code...
        features = {}

        # Numeric feature extraction with more granular scaling
        fitness_map = {
            'Couch potato 🛋': 0.2,
            'Average, but could be better 🚶': 0.5,
            'Pretty active 💪': 0.8,
            'Athlete level 🏃‍♀️': 1.0
        }
        weather_map = {
            'No way, I\'ll reschedule 🛌': 0.2,
            'Maybe if it\'s just cloudy ☁️': 0.5,
            'Rain won\'t stop me! 🌧': 1.0
        }
        experience_map = {
            'Yes, a few times': 0.6,
            'No, but I\'d like to try': 0.3,
            'Yes, and I hike regularly': 1.0,
            'Not my thing': 0.1
        }

        # Extract and normalize features
        features.update({
            'fitness_level': fitness_map.get(user_response.get('fitness_level'), 0.5),
            'weather_tolerance': weather_map.get(user_response.get('weather_preference'), 0.5),
            'experience': experience_map.get(user_response.get('hiking_experience'), 0.3),
            'hiking_likelihood': int(user_response.get('hiking_likelihood', 3)) / 5.0,
            'preferred_length': self._get_preferred_length(user_response.get('trail_length_preference', [])),
            'social_preference': self._get_social_preference(user_response.get('hiking_companion')),
            'adventure_score': self._get_adventure_score(user_response)
        })

        return np.array(list(features.values()))

    def _get_preferred_length(self, length_preferences):
        """Calculate preferred length score."""
        if not isinstance(length_preferences, list):
            length_preferences = [length_preferences]

        score = 0
        for pref in length_preferences:
            if 'Short' in pref:
                score += 0.33
            elif 'nice challenge' in pref:
                score += 0.66
            elif 'Long' in pref:
                score += 1.0
        return score / max(len(length_preferences), 1)

    def _get_social_preference(self, companion_preference):
        """Calculate social preference score."""
        social_scores = {
            'Alone for some peace and quiet': 0.2,
            'With friends or a group': 0.8,
            'Family trips': 0.6,
            'My pet 🐕': 0.4
        }
        return social_scores.get(companion_preference, 0.5)

    def _get_adventure_score(self, user_response):
        """Calculate adventure preference score."""
        adventure_score = 0

        if 'Going on adventures' in user_response.get('weekend_preference', ''):
            adventure_score += 0.4
        if 'A snowy mountain 🏔' in user_response.get('dream_destination', ''):
            adventure_score += 0.3
        if 'Pumped-up workout beats 🎧' in user_response.get('music_preference', ''):
            adventure_score += 0.3

        return adventure_score

    def get_recommendations(self, user_response, n_recommendations=3):
        """Get personalized trail recommendations using hybrid model."""
        user_features = self.process_user_preferences(user_response)

        # Prepare trail features
        trail_features = np.hstack([
            self.trails_df[['Length', 'Difficulty_Score', 'Average_rating']].values,
            self.tag_embeddings
        ])

        # If models haven't been built yet, build them
        if self.user_model is None:
            self._build_neural_models(
                user_features.shape[0],
                trail_features.shape[1]
            )

        # Get user embedding
        user_embedding = self.user_model.predict(
            user_features.reshape(1, -1),
            verbose=0
        )

        # Get trail embeddings
        trail_embeddings = self.trail_model.predict(trail_features, verbose=0)

        # Calculate similarity scores
        similarity_scores = cosine_similarity(user_embedding, trail_embeddings)[0]

        # Get hybrid scores
        user_features_repeated = np.tile(
            user_features.reshape(1, -1),
            (len(self.trails_df), 1)
        )
        hybrid_scores = self.hybrid_model.predict(
            [user_features_repeated, trail_features],
            verbose=0
        ).flatten()

        # Combine similarity and hybrid scores
        rating_scores = self.trails_df['Average_rating'].values / 5.0
        final_scores = (
            0.3 * similarity_scores +
            0.4 * hybrid_scores +
            0.3 * rating_scores
        )

        # Adjust scores based on trail length preferences
        preferred_tags = user_response.get('trail_length_preference', [])
        for i, trail_length in enumerate(self.trails_df['Length']):
            if trail_length <= 5:
                length_adjustment = 1.2 if 'Short' in preferred_tags else 0.9
            elif 5 < trail_length <= 10:
                length_adjustment = 1.2 if 'Medium' in preferred_tags else 0.9
            else:
                length_adjustment = 1.2 if 'Long' in preferred_tags else 0.8
            final_scores[i] *= length_adjustment

        # Get top recommendations
        recommendations_idx = np.argsort(final_scores)[::-1][:n_recommendations]
        recommendations = self.trails_df.iloc[recommendations_idx].copy()
        recommendations['Match_Score'] = final_scores[recommendations_idx]

        return recommendations

def format_recommendations(recommendations):
    """Format recommendations with detailed explanation."""
    output = []

    for _, trail in recommendations.iterrows():
        match_percentage = int(trail['Match_Score'] * 100)

        output.append(
            # f"🎯 Trail Match: {trail['Trail_name']} ({match_percentage}% match)\n"
            f"📍 Location: {trail['Location']}\n"
            f"💪 Difficulty: {trail['Difficulty']} ({trail['Difficulty_Score']:.2f})\n"
            f"📏 Length: {trail['Length']:.1f} km\n"
            f"⭐ Rating: {trail['Average_rating']:.1f}/5.0\n"
            f"🏷️ Features: {trail['Tags']}\n"
            f"💡 Why this trail: This trail matches your preferences for "
            f"{'difficulty level' if trail['Difficulty_Score'] > 0.5 else 'easier trails'}, "
            f"{'longer distances' if trail['Length'] > 7 else 'moderate distances' if trail['Length'] > 3 else 'shorter distances'}, "
            f"and includes features you mentioned enjoying.\n"
        )

    return "\n".join(output)

In [143]:
# Sample user preferences
sample_user = {
    'fitness_level': 'Pretty active 💪',
    'weather_preference': 'Rain won\'t stop me! 🌧',
    'hiking_experience': 'Yes, and I hike regularly',
    'hiking_likelihood': 4,
    'trail_length_preference': ['A nice challenge (5-8 km)'],
    'hiking_companion': 'With friends or a group',
    'weekend_preference': 'Going on adventures',
    'dream_destination': 'A snowy mountain 🏔',
    'music_preference': 'Pumped-up workout beats 🎧'
}

# Initialize and use the recommender
def demo_trail_recommender():
    # Initialize recommender
    recommender = AdvancedTrailRecommender(embedding_dim=32)

    # Preprocess trail data
    recommender.preprocess_trails(df)

    # Get recommendations
    recommendations = recommender.get_recommendations(sample_user, n_recommendations=3)

    # Format and display recommendations
    print("🏃‍♂️ Trail Recommendations for Active Hiker Profile:")
    print("\nUser Profile:")
    print(f"- Fitness Level: {sample_user['fitness_level']}")
    print(f"- Hiking Experience: {sample_user['hiking_experience']}")
    print(f"- Preferred Length: {sample_user['trail_length_preference'][0]}")
    print(f"- Weather Tolerance: {sample_user['weather_preference']}")
    print("\n" + "="*50 + "\n")
    print(format_recommendations(recommendations))

# Run the demo
if __name__ == "__main__":
    demo_trail_recommender()

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
🏃‍♂️ Trail Recommendations for Active Hiker Profile:

User Profile:
- Fitness Level: Pretty active 💪
- Hiking Experience: Yes, and I hike regularly
- Preferred Length: A nice challenge (5-8 km)
- Weather Tolerance: Rain won't stop me! 🌧


📍 Location: Junnar, Maharashtra, India
💪 Difficulty: Hard (1.00)
📏 Length: 6.6 km
⭐ Rating: 5.0/5.0
🏷️ Features: Hiking, Walking, Forest, River, Views, Wildflowers, Wildlife, Rocky, Muddy
💡 Why this trail: This trail matches your preferences for difficulty level, moderate distances, and includes features you mentioned enjoying.

📍 Location: Chikkaballapur, Karnataka, India
💪 Difficulty: Hard (1.00)
📏 Length: 7.1 km
⭐ Rating: 4.7/5.0
🏷️ Features: Hiking, Walking, Views, Historic site, Rocky, Bugs
💡 Why this trail: This trail matches your preferences for difficulty level, longer distances, and includes features you mentioned enjoying.

📍 Location: Khaltse, Ladakh, India
💪 Difficulty: Hard (1.00)
📏 Length: 9.5 km
⭐ 